In [10]:
import requests
import sys
import parsel
sys.path.append('..')
from configure.settings import DBSelector
import pandas as pd

In [2]:
url = 'https://www.jisilu.cn/data/cbnew/adj_logs/?bond_id={}&adj_type=D'

In [5]:
engine = DBSelector().get_engine('db_stock')

In [7]:
df = pd.read_sql('tb_bond_jisilu', con=engine)

In [8]:
df.head()

,可转债代码,可转债名称,bond_py,可转债价格,可转债涨幅,正股代码,正股名称,stock_py,正股现价,正股涨跌幅,...,after_next_put_dt,标记,is_min_price,blocked,putting,notes,volatility_rate,debt_rate,是否两融标的,更新日期
0,113600,新星转债,xxzz,111.444,-1.05,603978,深圳新星,szxx,11.32,3.57,...,1,{'M': ''},0,0,N,None,buy,buy,是,2024-09-05 15:10
1,113050,南银转债,nyzz,118.505,-0.34,601009,南京银行,njyh,10.19,0.39,...,0,{'M': ''},0,0,N,None,buy,buy,是,2024-09-05 15:10
2,111017,蓝天转债,ltzz,146.487,0.11,605368,蓝天燃气,ltrq,13.62,0.07,...,0,{'G': '本次（2024年3月18日至2024年4月9日）不提前赎回，以2024年10月...,0,0,N,None,buy,buy,是,2024-09-05 15:10
3,110048,福能转债,fnzz,185.134,-1.16,600483,福能股份,fngf,9.75,-0.91,...,1,"{'G': '本次不提前赎回，在剩余存续期内(至2024年12月6日）均不提前赎回', 'M...",0,0,N,None,buy,buy,是,2024-09-05 15:10
4,110083,苏租转债,szzz,148.237,0.28,600901,江苏金租,jsjz,4.51,0.00,...,0,{'M': ''},0,0,N,None,buy,buy,是,2024-09-05 15:10


In [9]:
code_list = df['可转债代码'].tolist()

In [10]:
len(code_list)

534

In [32]:
def get_html(bond_id):
    url = 'https://www.jisilu.cn/data/cbnew/adj_logs/?bond_id={}&adj_type=D'.format(bond_id)
    r = requests.get(url)
    data = r.text
    r.encoding = 'utf-8'
    return data

html = get_html('113600')

In [35]:
rows = parsel.Selector(html).xpath('//tbody//tr')

In [36]:
rows

[<Selector query='//tbody//tr' data='<tr><td>新星转债</td><td>2024-08-02</td><...'>,
 <Selector query='//tbody//tr' data='<tr><td>新星转债</td><td>2023-06-30</td><...'>]

In [39]:
rows[1].xpath('//td[2]/text()').extract_first()

'2024-08-02'

In [40]:


def parse_html(html, code):
    rows = parsel.Selector(html).xpath('//tbody/tr')

    result = []
    for row in rows:
        try:
            bond_name = row.xpath('./td[1]/text()').extract_first()
            bond_adjust_date = row.xpath('./td[2]/text()').extract_first()
            bond_adjust_previous_price = float(row.xpath(
                './td[3]/text()').extract_first())
            bond_adjust_price = float(row.xpath('./td[4]/text()').extract_first())
            bond_adjust_executed_date = row.xpath('./td[5]/text()').extract_first()
            bond_adjust_buttom_price = float(row.xpath('./td[6]/text()').extract_first())
            adjust_percent = (bond_adjust_previous_price - bond_adjust_price)/bond_adjust_previous_price*100
            buttom_adjust_percent = (bond_adjust_previous_price - bond_adjust_buttom_price)/bond_adjust_previous_price*100

            data_dict = {
                'code': code, 'bond_name': bond_name, 'bond_adjust_date': bond_adjust_date, 
                'bond_adjust_previous_price': bond_adjust_previous_price, 'bond_adjust_price': bond_adjust_price, 
                'bond_adjust_executed_date': bond_adjust_executed_date, 'bond_adjust_buttom_price': bond_adjust_buttom_price,
                'adjust_percent': adjust_percent,
                'buttom_adjust_percent':buttom_adjust_percent,
                'delta':buttom_adjust_percent - adjust_percent
            }
            result.append(data_dict)
        except Exception as e:
            print(e)
            print(code)

    return result

def dump_mongodb(data):
    client =DBSelector().mongo()
    db = client['db_stock']
    collection = db['tb_bond_jisilu_adjust']
    collection.insert_many(data)
        
# adjust_data = parse_html(html,'118040')
# if len(adjust_data) > 0:
#     dump_mongodb(adjust_data)

In [15]:
html

'<table class="tablesorter" style="width:440px;"><thead><tr><th>转债名称</th><th>股东大会日</th><th>下修前<br>转股价</th><th>下修后<br>转股价</th><th>新转股价<br>生效日期</th><th>下修底价</th></tr></thead><tbody><tr><td>宏微转债</td><td>2024-01-29</td><td>62.350</td><td>40.000</td><td>2024-02-01</td><td>37.010</td></tr></tbody></table>'

In [41]:
for code in code_list:
    html = get_html(code)
    adjust_data = parse_html(html,code)
    if len(adjust_data) > 0:
        dump_mongodb(adjust_data)

could not convert string to float: '股东大会未通过'
110060
could not convert string to float: '股价上涨，股东大会取消'
113619
could not convert string to float: '股东大会未通过'
123238
could not convert string to float: '股东大会未通过'
127051
float() argument must be a string or a real number, not 'NoneType'
110058
could not convert string to float: '股东大会未通过'
113676
could not convert string to float: '股东大会未通过'
113678
could not convert string to float: '股东大会不通过'
118037
could not convert string to float: '股东大会未通过'
111016
could not convert string to float: '股东大会未通过'
123052
could not convert string to float: '股东大会未通过'
123183
could not convert string to float: '股东大会未通过'
123183
could not convert string to float: '股东大会未通过'
123196
could not convert string to float: '股东大会未通过'
113059


In [11]:
def fetch_data_from_mongo():
    client = DBSelector().mongo()
    db = client['db_stock']
    collection = db['tb_bond_jisilu_adjust']
    data = collection.find()
    df = pd.DataFrame(data)
    return df

In [12]:
df = fetch_data_from_mongo()

In [4]:
len(df)

222

In [5]:
df.head()

,_id,code,bond_name,bond_adjust_date,bond_adjust_previous_price,bond_adjust_price,bond_adjust_executed_date,bond_adjust_buttom_price,adjust_percent,buttom_adjust_percent,delta
0,66d9dff1c19667038f98018c,113600,新星转债,2024-08-02,17.45,10.00,2024-08-06,9.24,42.693410,47.048711,4.355301
1,66d9dff1c19667038f98018d,113600,新星转债,2023-06-30,23.85,17.45,2023-07-04,17.45,26.834382,26.834382,0.000000
2,66d9dff2c19667038f98018f,113033,利群转债,2024-05-06,6.80,4.90,2024-05-08,4.89,27.941176,28.088235,0.147059
3,66d9dff3c19667038f980191,118026,利元转债,2024-09-02,45.00,21.00,2024-09-04,19.09,53.333333,57.577778,4.244444
4,66d9dff3c19667038f980192,118026,利元转债,2023-12-01,124.62,45.00,2023-12-05,43.49,63.890226,65.101910,1.211684


In [13]:
df['delta_percent']=(df['bond_adjust_price']-df['bond_adjust_buttom_price'])/df['bond_adjust_previous_price'] * 100

In [9]:
df.head()

,_id,code,bond_name,bond_adjust_date,bond_adjust_previous_price,bond_adjust_price,bond_adjust_executed_date,bond_adjust_buttom_price,adjust_percent,buttom_adjust_percent,delta,delta_percent
0,66d9dff1c19667038f98018c,113600,新星转债,2024-08-02,17.45,10.00,2024-08-06,9.24,42.693410,47.048711,4.355301,4.355301
1,66d9dff1c19667038f98018d,113600,新星转债,2023-06-30,23.85,17.45,2023-07-04,17.45,26.834382,26.834382,0.000000,0.000000
2,66d9dff2c19667038f98018f,113033,利群转债,2024-05-06,6.80,4.90,2024-05-08,4.89,27.941176,28.088235,0.147059,0.147059
3,66d9dff3c19667038f980191,118026,利元转债,2024-09-02,45.00,21.00,2024-09-04,19.09,53.333333,57.577778,4.244444,4.244444
4,66d9dff3c19667038f980192,118026,利元转债,2023-12-01,124.62,45.00,2023-12-05,43.49,63.890226,65.101910,1.211684,1.211684


In [14]:
new_sort_df = df.sort_values(by='delta_percent',ascending=False)

In [13]:
new_sort_df.to_excel('sort_delta_df.xlsx')

In [14]:
new_sort_df.head()

,_id,code,bond_name,bond_adjust_date,bond_adjust_previous_price,bond_adjust_price,bond_adjust_executed_date,bond_adjust_buttom_price,adjust_percent,buttom_adjust_percent,delta,delta_percent
221,66d9e07fc19667038f980310,118000,嘉元转债,2024-07-04,50.47,41.88,2024-07-11,10.47,17.020012,79.255003,62.234991,62.234991
219,66d9e074c19667038f98030c,113601,塞力转债,2024-09-05,16.98,13.71,2024-09-09,6.73,19.257951,60.365135,41.107185,41.107185
5,66d9dff3c19667038f980193,118026,利元转债,2023-06-02,218.59,174.87,2023-06-06,90.96,20.000915,58.387849,38.386934,38.386934
220,66d9e079c19667038f98030e,118008,海优转债,2024-07-11,217.30,109.82,2024-07-15,29.82,49.461574,86.277036,36.815462,36.815462
197,66d9e05ec19667038f9802e2,113610,灵康转债,2024-07-12,8.51,8.00,2024-07-16,4.89,5.992949,42.538190,36.545241,36.545241


In [15]:
new_sort_df['acutal_delta'] = new_sort_df['delta']/new_sort_df['buttom_adjust_percent'] * 100

In [16]:
new_sort_df.sort_values(by='acutal_delta',ascending=False).head()

,_id,code,bond_name,bond_adjust_date,bond_adjust_previous_price,bond_adjust_price,bond_adjust_executed_date,bond_adjust_buttom_price,adjust_percent,buttom_adjust_percent,delta,delta_percent,acutal_delta
77,66d9e010c19667038f98020f,123056,雪榕转债,2024-07-15,4.20,4.15,2024-07-16,2.66,1.190476,36.666667,35.476190,35.476190,96.753247
76,66d9e010c19667038f98020e,123056,雪榕转债,2024-08-23,4.15,4.10,2024-08-26,3.07,1.204819,26.024096,24.819277,24.819277,95.370370
186,66d9e053c19667038f9802ce,110047,山鹰转债,2024-07-08,2.37,2.32,2024-07-10,1.47,2.109705,37.974684,35.864979,35.864979,94.444444
129,66d9e02cc19667038f98026c,110063,鹰19转债,2024-07-08,2.37,2.32,2024-07-10,1.47,2.109705,37.974684,35.864979,35.864979,94.444444
185,66d9e053c19667038f9802cd,110047,山鹰转债,2024-08-15,2.32,2.25,2024-08-19,1.44,3.017241,37.931034,34.913793,34.913793,92.045455


In [21]:
to_end_bond = new_sort_df[(new_sort_df['bond_adjust_executed_date']>'2024-01-01')&(new_sort_df['acutal_delta']<=10)]

In [16]:
to_end_bond.to_excel('to_end_bond1.xlsx')

In [5]:
str = '''

110060 天路转债 原因: '股东大会未通过'

113619 世运转债 原因: '股价上涨，股东大会取消'

123238 卡倍转02 原因: '股东大会未通过'

127051 博杰转债 原因: '股东大会未通过'

113676 荣23转债 原因: '股东大会未通过'

113678 中贝转债 原因: '股东大会未通过'

118037 上声转债 原因: '股东大会不通过'

111016 神通转债 原因: '股东大会未通过'

123052 飞鹿转债 原因: '股东大会未通过'

123183 海顺转债 原因: '股东大会未通过'

123196 正元转02 原因: '股东大会未通过'

113059 福莱转债 原因: '股东大会未通过'



'''

In [6]:
print(str)


110060 原因: '股东大会未通过'

113619 原因: '股价上涨，股东大会取消'

123238 原因: '股东大会未通过'

127051 原因: '股东大会未通过'

113676 原因: '股东大会未通过'

113678 原因: '股东大会未通过'

118037 原因: '股东大会不通过'

111016 原因: '股东大会未通过'

123052 原因: '股东大会未通过'

123183 原因: '股东大会未通过'

123183 原因: '股东大会未通过'

123196 原因: '股东大会未通过'

113059 原因: '股东大会未通过'



In [7]:
new_sort_df.to_excel('sort_delta_df_all.xlsx')

NameError: name 'new_sort_df' is not defined

In [8]:
new_sort_df

NameError: name 'new_sort_df' is not defined

In [15]:
to_end_bond = new_sort_df[(new_sort_df['bond_adjust_executed_date']>'2024-01-01')]